# Propagating Knowledge in an Heterogeneous Network


In this short tutorial, we briefly discuss the problem of propagating knowledge (in other terms, label information) in a network, starting from a limited number of labeled nodes.

We assume the network can be formalized as an undirected multi-graph $G = (V, E, r)$ where:
* $V$ is a set of *nodes*, or *vertices* (e.g. $V = \{ Mark, John, Paul, Lucas \}$ ).
* $E$ is a set of *edges* (e.g. $E = \{ friendOf, worksWith \}$).
* $r : E \rightarrow \mathcal P(\{ \{ v_{i}, v_{j} \} \mid v_{i}, v_{j} \in V \})$ is a function that assigns, to each edge, a set of unordered vertex pairs (e.g. $r(friendOf) = \{ \{ Mark, Paul \}, \{ Mark, John \} \}$).

In particular, we assume all nodes represent *entities* in a domain (such as persons in a social network, or articles and authors in a bibliographic citation network), while edges denote *relations* that hold among such entities (such as friendship in a social network, or co-authorship in a bibliographic citation network).

The term *heterogeneous* refers to the fact that, in this context, entities can be interlinked by an heterogeneous set of relations, each holding a different meaning.

We assume some relations are **homophilic**, i.e. they tend to link entities that are similar w.r.t. some aspects. For instance, we know that friends in social networks tend to have common characteristics, such as religious beliefs and political views [1, 2].

For such a reason, we focus on the problem of *propagating* knowledge, or label information, in heterogeneous networks: from a low number of *labeled examples* (i.e. entities which a specific property, e.g. *Christian*) we aim at *propagating* the knowledge about such property across the whole network (e.g. by following friendship relations), associating a varying degree of belief to each entity.

We can associate a binary label $f_{i} \in \{ +1, -1 \}$ to each entity $v_{i} \in V$, where $f_{i} = 1$ if the property holds (e.g. the person is Christian), and $f_{i} = -1$ if the property does not hold (e.g. the person is not Christian).

The task of propagating knowledge across all entities in the network can be cast as finding a labeling $f \in \{ +1, -1 \}^{|V|}$ to all entities in the network, such that *similar entities are likely to share the same label* [3]. In this context, similarity information between entities is provided by the relationships holding between them (e.g. if two persons are friends, they might be more likely to be both Christian or non-Christian).

Following [4], we can formulate a *cost function* over possible labelings $f$, which penalizes (i.e. assigns an higher cost to) labelings that assign different labels to similar entities, and favors (i.e. assigns a lower cost to) labelings that assign the same labels to similar entities.

In particular, we can formulate the following *quadratic function*:

$$E(f) = \frac{1}{2} \sum_{v_{i} \in V} \sum_{v_{j} \in V} W_{ij} (f_{i} - f_{j})^{2},$$

where $W \in \mathbb{R}^{|V| \times |V|}$ is a *similarity matrix* such that $W_{ij} = W_{ji} > 0$ if entities $v_{i}$ and $v_{j}$ are *similar*, and 0 otherwise.

The cost function works as follows. Imagine $v_{i}$ and $v_{j}$ are considered similar: it follows that $W_{ij} > 0$, and thus the cost associated to such a labeling will be $W_{ij}$ if $f_{i} \neq f_{j}$, and $0$ if $f_{i} = f_{j}$.

Now, assume $L \subset V$ denotes *labeled entities* (i.e. entities for which we know the label), and denote as $y_{i} \in \{ +1, -1 \}$ the label for the $i$-th entity. Finding an optimal labeling for all entities in $V$ can be cast as the following optimization problem:

$$\text{minimize} \; \; \;  E(f) \; \; \; s.t. \; \; \;f \in \{ -1, 1 \}^{|V|}, \; \; \;\forall v_{i} \in L : f_{i} = y_{i}.$$


However, there are two problems with the optimization problem above.
1. In the multi-label (non-binary) case, minimizing the cost function is NP-hard.
2. Binary (or discrete) labels do not express the *uncertainty* associated to a labeling.

For such reasons, in [4] authors propose the following solution: *relax* discrete labels $f \in \{ +1, -1 \}^{|V| \times |V|}$ into continuous labels $f \in \mathbb{R}^{|V| \times |V|}$.

Thus the optimization problem above can be reformulated as follows:

$$\text{minimize} \; \; \;  E(f) + \epsilon \sum_{v_{i} \in V} f_{i}^{2} \; \; \; s.t. \; \; \;f \in \mathbb{R}^{|V|}, \; \; \;\forall v_{i} \in L : f_{i} = y_{i},$$

where $\epsilon \in \mathbb{R}_{+}$ weights a regularization term that favors smaller values of $f$. This is a constrained quadratic optimization problem with a *closed form* solution, which can be calculated efficiently.

In partular, the optimal labeling $f^{*}$ is given by $f^{f_{L}^{*}, f_{U}^{*}}$, where $f_{L}^{*} = y_{L}$ (i.e. $f_{L}$ coincides with training labels), and $f_{U}^{*}$ is given by:

$$f_{U}^{*} = (L_{UU} + \epsilon I)^{-1} W_{UL} f_{L},$$

where $L = D - W$, and $D$ is a diagonal matrix such that $D_{ii} = \sum_{ij} W_{ij}$.



[1] McPherson, M., Lovin, L.S., Cook, J.M.: Birds of a Feather: Homophily in Social Networks. Annual Review of Sociology 27 (1), 415–444 (2001)

[2] 2. Aggarwal, C.C. (ed.): Social Network Data Analytics. Springer (2011)

[3] Chapelle, O., Schölkopf, B., Zien, A. (eds.): Semi-Supervised Learning. MIT Press (2006)

[4] Zhu, X., Ghahramani, Z., Lafferty, J.D.: Semi-Supervised Learning Using Gaussian Fields and Harmonic Functions. In: T. Fawcett, et al. (eds.) Proceedings of ICML’03, pp. 912–919. AAAI Press (2003)


In [26]:
import numpy as np
import theano
import theano.tensor as T
import theano.tensor.nlinalg as nlinalg

# Theano symbolic formula for calculating, given { f, l, W, eps }:
# f* = (L_UU + epsI)^-1 W_UL f_L
def propagation(f, l, R, mu, eps):
    # The similarity matrix W is a linear combination of the slices in R
    W = T.tensordot(R, mu, axes=1)

    # The following indices correspond to labeled and unlabeled examples
    labeled = T.eq(l, 1).nonzero()
    unlabeled = T.eq(l, 0).nonzero()

    # Calculating the graph Laplacian of W
    D = T.diag(W.sum(axis=0))
    L = D - W

    # Computing L_UU (the Laplacian over unlabeled examples)
    L_UU = L[unlabeled][:, unlabeled][:, 0, :]

    # Computing the inverse of the (regularized) Laplacian iA = (L_UU + epsI)^-1
    epsI = eps * T.eye(L_UU.shape[0])
    rL_UU = L_UU + epsI
    iA = nlinalg.matrix_inverse(rL_UU)

    # Computing W_UL (the similarity matrix between unlabeled and labeled examples)
    W_UL = W[unlabeled][:, labeled][:, 0, :]
    f_L = f[labeled]

    # f* = (L_UU + epsI)^-1 W_UL f_L
    f_star = iA.dot(W_UL.dot(f_L))

    return f_star

# Training labels, similarity matrix and weight of the regularization term
f, R, mu, eps = T.dvector('f'), T.dtensor3('R'), T.dvector('mu'), T.dscalar('eps')

# Indices of labeled examples
l = T.ivector('l')

f_star = propagation(f, l, R, mu, eps)

propagation_function = theano.function([f, l, R, mu, eps], f_star, on_unused_input='ignore')

In [66]:
from IPython.core.display import HTML
import d3_lib
import random

nb_nodes = 32

R = np.zeros((nb_nodes, nb_nodes, 1))
even_edges = [(i, i + 2) for i in range(0, nb_nodes, 2) if (i + 2) < nb_nodes]
odd_edges = [(i, i + 2) for i in range(1, nb_nodes, 2) if (i + 2) < nb_nodes]

all_edges = even_edges + odd_edges

for source, target in all_edges:
    R[source, target, 0], R[target, source, 0] = 1.0, 1.0

mu, eps = np.ones(1), 1e-4
y = np.array([+ 1.0, - 1.0] + ([.0] * (nb_nodes - 2)))
l, u = np.array(y != 0, dtype='int8'), np.array(y == 0, dtype='int8')

f = y
print(f)

graph = {"nodes": [], "links": []}

for i in range(nb_nodes):
    group = 1 if f[i] > 1e-1 else 2 if f[i] < -1e-1 else 3
    graph["nodes"].append( {"name": "i" + str(i), "group": int(group)} )

for i in range(nb_nodes):
    for j in range(nb_nodes):
        if (i, j) in all_edges:
            graph["links"].append( {"source": i, "target": j, "value": 1} )

# visualize as force-directed graph in D3
HTML(d3_lib.set_styles(['force_directed_graph']) + \
     '<script src="lib/d3/d3.min.js"></script>' + \
     d3_lib.draw_graph('force_directed_graph', {'data': graph}))

[ 1. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


In [65]:
_f = propagation_function(y, l, R, mu, eps)
f[u.nonzero()] = _f
print(f)

graph = {"nodes": [], "links": []}

for i in range(nb_nodes):
    group = 1 if f[i] > 1e-1 else 2 if f[i] < -1e-1 else 3
    graph["nodes"].append( {"name": "i" + str(i), "group": int(group)} )

for i in range(nb_nodes):
    for j in range(nb_nodes):
        if (i, j) in all_edges:
            graph["links"].append( {"source": i, "target": j, "value": 1} )

# visualize as force-directed graph in D3
HTML(d3_lib.set_styles(['force_directed_graph']) + \
     '<script src="lib/d3/d3.min.js"></script>' + \
     d3_lib.draw_graph('force_directed_graph', {'data': graph}))

[ 1.         -1.          0.99851228 -0.99851228  0.99712441 -0.99712441
  0.99583626 -0.99583626  0.99464769 -0.99464769  0.99355858 -0.99355858
  0.99256883 -0.99256883  0.99167834 -0.99167834  0.99088701 -0.99088701
  0.99019478 -0.99019478  0.98960156 -0.98960156  0.9891073  -0.9891073
  0.98871196 -0.98871196  0.98841548 -0.98841548  0.98821785 -0.98821785
  0.98811904 -0.98811904]
